In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("NYCTaxiFares.csv")

In [4]:
df.head()

,pickup_datetime,fare_amount,fare_class,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2010-04-19 08:17:56 UTC,6.5,0,-73.992365,40.730521,-73.975499,40.744746,1
1,2010-04-17 15:43:53 UTC,6.9,0,-73.990078,40.740558,-73.974232,40.744114,1
2,2010-04-17 11:23:26 UTC,10.1,1,-73.994149,40.751118,-73.960064,40.766235,2
3,2010-04-11 21:25:03 UTC,8.9,0,-73.990485,40.756422,-73.971205,40.748192,1
4,2010-04-17 02:19:01 UTC,19.7,1,-73.990976,40.734202,-73.905956,40.743115,1


In [5]:
y_label = df["fare_amount"].values

In [7]:
p_longitude = df["pickup_longitude"].values
p_latitude = df["pickup_latitude"].values
d_longitude = df["dropoff_longitude"].values
d_latitude = df["dropoff_latitude"].values
passenger_count = df["passenger_count"].values

In [8]:
X = np.stack([p_longitude,p_latitude,d_longitude,d_latitude,passenger_count],axis = 1)

In [13]:
y_label = y_label.reshape((-1,1))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y_label)

In [18]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [23]:
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [24]:
import torch.nn as nn

In [45]:
class Model(nn.Module):
    def __init__(self, in_features=5, h1=100, h2=300, h3 = 200, out_features=1):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.fc3 = nn.Linear(h2,h3)
        self.out = nn.Linear(h3, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x

In [46]:
model = Model()

In [47]:
criterion = nn.MSELoss()  # we'll convert this to RMSE later
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [48]:
epochs = 100

for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred,y_train)
    
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')
    
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  1  loss: 73.39455414
epoch: 11  loss: 61.91926956
epoch: 21  loss: 56.44284821
epoch: 31  loss: 56.58450317
epoch: 41  loss: 56.49511337
epoch: 51  loss: 56.34593201
epoch: 61  loss: 56.36091232
epoch: 71  loss: 56.34038925
epoch: 81  loss: 56.33868027
epoch: 91  loss: 56.33785629


In [56]:
with torch.no_grad():
    y_val = model.forward(X_test)
    loss = criterion(y_val,y_test)
    print(str(loss))

tensor(56.1011)
